Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [348]:
from itertools import combinations
from collections import deque
import numpy as np
import random
from geopy.distance import geodesic
import networkx as nx

import pandas as pd
from icecream import ic
import logging
import pickle

logging.basicConfig(level=logging.DEBUG)

# Initialization

In [349]:
COUNTRY = 'italy'
# Read the CSV file
CITIES = pd.read_csv(f'cities/{COUNTRY}.csv', header=None, names = ['name', 'lat', 'lon'])

# Calculate distances between all cities
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic((c1.lat, c1.lon), (c2.lat, c2.lon)).km

DIST_MATRIX[0, 1]

np.float64(349.30401144305745)

# Helper Functions

In [350]:
def tsp_cost(tsp: list[int]) -> float:
    # Check that the TSP is a cycle and that all cities have been covered
    assert tsp[0] == tsp[-1], f"first city is #{tsp[0]}, last city is #{tsp[-1]}"
    # logging.debug(f"set(tsp): {set(tsp)}\nset(range(len(tsp))): {set(range(len(tsp)))}")
    assert set(tsp) == set(range(len(tsp) - 1)), f"tsp covers {len(tsp)}, should cover {len(tsp) - 1}"

    # Finally, compute the total cost
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]

    return tot_cost

def print_tsp(tsp: list):
    first = True
    for city in tsp:
        if first:
            prev_city = city
            first = False
            continue

        logging.info(f"step: {CITIES.at[prev_city,'name']} -> {CITIES.at[city,'name']} ({DIST_MATRIX[prev_city, city]:.2f}km)")
        prev_city = city

    logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")



# Segments

In [351]:
def tsp_segment_to_series(tsp: list[tuple]) -> list[int]:
    # Convert the list of segments to a list of cities
    tsp_series = [t[0] for t in tsp]
    tsp_series.append(tsp[0][0])
    return tsp_series


# def print_tsp_segments(tsp: list):
#     # logging.info(f"Total cost: {tsp_cost(tsp):.2f}km")

#     for segment in tsp:
#         city1, city2 = segment
#         logging.info(f"step: {CITIES.at[city1,'name']} -> {CITIES.at[city2,'name']} ({DIST_MATRIX[city1, city2]:.2f}km)")


def complete_segment_tsp(tsp: list[tuple], degrees: list[int]) -> list[tuple]:
    assert len(degrees) == len(CITIES), f"len(degrees)={len(degrees)} != len(CITIES)={len(CITIES)}"
    assert all (degree > 0 for degree in degrees), f"degrees={degrees} should be all > 0"

    indices_with_value_1 = [index for index, value in enumerate(degrees) if value == 1]
    assert len(indices_with_value_1) == 2, f"len(indices_with_value_1)={len(indices_with_value_1)} != 2"
    tsp.append((indices_with_value_1[0], indices_with_value_1[1]))

    return tsp

def order_segments(segments: list[tuple]) -> list[tuple]:
    # Create a dictionary to map each node to its connected nodes
    connections = {}
    for u, v in segments:
        if u in connections:
            connections[u].append(v)
        else:
            connections[u] = [v]
        if v in connections:
            connections[v].append(u)
        else:
            connections[v] = [u]

    # Start with the first segment in the list
    ordered_segments = [segments[0]]
    current = segments[0][1]

    # Reorder the list by following the connections
    while len(ordered_segments) < len(segments):
        next_city = connections[current].pop()
        connections[next_city].remove(current)
        next_segment = (current, next_city)
        ordered_segments.append(next_segment)
        current = next_city

    return ordered_segments


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

## First Greedy Algorithm: Taking Closest While Traversing

In [352]:
# # Iterate over the rows and log information
# for index, row in cities.iterrows():
#     logging.debug(f'Index: {index}: {row["name"]} ({row.lat}, {row.lon})')
#     if index == 10:
#         break

# Try subset of rows
CITIES.iloc[5:10]

,name,lat,lon
5,Bolzano,46.50,11.35
6,Brescia,45.55,10.22
7,Cagliari,39.22,9.10
8,Catania,37.50,15.08
9,Ferrara,44.84,11.61


In [353]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True

tsp = list()
tsp.append(int(city))

while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)")
    
    visited[closest] = True
    city = closest
    tsp.append(int(city))
    
logging.debug(f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)")
tsp.append(tsp[0])

tot_cost = tsp_cost(tsp)
logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")

with open(f'{COUNTRY}_tsp_greedy1.pkl', 'wb') as file:
    pickle.dump(tsp, file)
tsp_greedy_1 = tsp

DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologna -> Modena (37.29km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94km)
DEBUG:root:step: Parma -> Piacenza (57.65km)
DEBUG:root:step: Piacenza -> Milan (60.65km)
DEBUG:root:step: Milan -> Monza (14.51km)
DEBUG:root:step: Monza -> Bergamo (33.92km)
DEBUG:root:step: Bergamo -> Brescia (46.02km)
DEBUG:root:step: Brescia -> Verona (61.42km)
DEBUG:root:step: Verona -> Vicenza (44.70km)
DEBUG:root:step: Vicenza -> Padua (30.13km)
DEBUG:root:step: Padua -> Venice (36.07km)
DEBUG:root:step: Venice -> Trieste (115.09km)
DEBUG:root:step: Trieste -> Bolzano (209.68km)
DEBUG:root:step: Bolzano -> Trento (49.94km)
DEBUG:root:step: Trento -> Novara (206.69km)
DEBUG:root:step: Novara -> Turin (84.46

## Second Greedy Algorithm: Taking Shortest Connections

In [354]:
# Define the segments as ((city1, city2), distance) nested tuples
all_segments = list(combinations(range(len(CITIES)), 2))
all_segments = [((i, j), DIST_MATRIX[i, j]) for i, j in all_segments]
# Then define the tsp as a list of segments and the union-find data structure
uf = nx.utils.UnionFind()
tsp = list()
cities_degrees = [0] * len(CITIES) # Degree as in how many connections/segments

all_segments_empty = False
while not all_segments_empty:
    # Extract (i.e. also remove) the shortest segment
    shortest_seg = min(all_segments, key=lambda x: x[1])
    all_segments.remove(shortest_seg)
    shortest_seg = shortest_seg[0]  # Remove the distance from the tuple

    # Check if the two cities are already connected AND if the vertices are peripheral
    different_sets = uf[shortest_seg[0]] != uf[shortest_seg[1]]
    peripheral_vertices = cities_degrees[shortest_seg[0]] < 2 and cities_degrees[shortest_seg[1]] < 2
    if different_sets and peripheral_vertices:
        # Update the uf and the tsp
        uf.union(*shortest_seg)         # * is the unpacking operator
        tsp.append(shortest_seg)
        # Update the degrees of the cities
        cities_degrees[shortest_seg[0]] += 1
        cities_degrees[shortest_seg[1]] += 1

    all_segments_empty = len(all_segments) == 0
    
# Close the cycle and reorder it 
tsp = complete_segment_tsp(tsp, cities_degrees)
tsp = order_segments(tsp)

# Convert the list of segments to a list of cities
tsp = tsp_segment_to_series(tsp)
print_tsp(tsp)

with open(f'{COUNTRY}_tsp_greedy2.pkl', 'wb') as file:
    pickle.dump(tsp, file)

tsp_greedy_2 = tsp


INFO:root:step: Giugliano in Campania -> Naples (11.15km)
INFO:root:step: Naples -> Salerno (46.25km)
INFO:root:step: Salerno -> Foggia (109.52km)
INFO:root:step: Foggia -> Andria (67.42km)
INFO:root:step: Andria -> Bari (50.18km)
INFO:root:step: Bari -> Taranto (77.63km)
INFO:root:step: Taranto -> Pescara (335.25km)
INFO:root:step: Pescara -> Syracuse (605.55km)
INFO:root:step: Syracuse -> Catania (51.23km)
INFO:root:step: Catania -> Reggio di Calabria (84.28km)
INFO:root:step: Reggio di Calabria -> Messina (12.48km)
INFO:root:step: Messina -> Palermo (192.10km)
INFO:root:step: Palermo -> Cagliari (390.28km)
INFO:root:step: Cagliari -> Sassari (173.89km)
INFO:root:step: Sassari -> Leghorn (344.66km)
INFO:root:step: Leghorn -> Prato (74.02km)
INFO:root:step: Prato -> Florence (17.17km)
INFO:root:step: Florence -> Trento (255.78km)
INFO:root:step: Trento -> Bolzano (49.94km)
INFO:root:step: Bolzano -> Trieste (209.68km)
INFO:root:step: Trieste -> Venice (115.09km)
INFO:root:step: Venice

# Evolutionary and Genetic Algorithms  
In the following, except for the last implementation which uses the Hypermodern Flow for Genetic Algorithms, all other instances will use the Modern Flow.  

#### Mutations Helper

In [355]:
def swap_mutation(individual):
    idx1, idx2 = random.sample(range(len(individual)), 2)
    individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

def scramble_mutation(individual, strength):
    num_elements = int(strength * len(individual))
    if num_elements <= 2:
        num_elements = 2

    indices = random.sample(range(len(individual)), num_elements)
    subset = [individual[i] for i in indices]
    random.shuffle(subset)
    for i, idx in enumerate(indices):
        individual[idx] = subset[i]
    return individual

def insert_mutation(individual):
    idx1, idx2 = random.sample(range(len(individual)), 2)
    gene = individual.pop(idx1)
    individual.insert(idx2, gene)
    return individual

def inversion_mutation(individual, strength):
    num_elements = int(strength * len(individual))
    if num_elements <= 2:
        num_elements = 2

    start_idx = random.randint(0, len(individual) - 1)

    # Create a circular buffer and rotate
    circular_buffer = deque(individual)
    circular_buffer.rotate(-start_idx)

    # Invert the selected number of elements
    subset = list(circular_buffer)[:num_elements]
    subset.reverse()
    for i in range(num_elements):
        circular_buffer[i] = subset[i]

    # Rotate back to the original position and revert to list
    circular_buffer.rotate(start_idx)
    individual[:] = list(circular_buffer)

    return individual

#### Crossover Helper

In [358]:
def cycle_crossover(parent1, parent2):
    size = len(parent1)
    child = [None] * size
    cycle_start = 0

    # Create one cycle. Save the genes of the cycle from parent 1
    complete = False
    while not complete:
        idx = cycle_start
        while True:
            # Copy the gene from parent1 to the child, then go on with the cycle exploration
            child[idx] = parent1[idx]
            idx = parent1.index(parent2[idx])
            # logging.debug(f"{child[idx]}) -> ")
            if idx == cycle_start:
                complete = True
                break
    
    # Fill in the remaining positions with genes from parent2
    for i in range(size):
        if child[i] is None:
            child[i] = parent2[i]
    
    return child

def inver_over_crossover(parent1, parent2):
    """ This implementation of the inver over refers to the basic version seen in https://www.sciencedirect.com/science/article/pii/S0898122111005530,
        but avoids the mutation chance and just mixes the parents, as I implement mutation separately."""
    idx1 = random.randint(0, len(parent1) - 1)
    city_i = parent1[idx1]
    idx2 = parent2.index(city_i)
    city_j = parent2[idx2 + 1] if idx2 < len(parent2) - 1 else parent2[0]
    logging.debug(f"city_i: {city_i} at position {idx1}, city_j: {city_j} at position {idx2}")
    
    # Perform inversion if city_i and city_j are not adjacent
    idx2 = parent1.index(city_j)
    if abs(idx1 - idx2) > 1:
        # Invert the section between the two cities
        if idx1 < idx2:
            parent1[idx1:idx2+1] = reversed(parent1[idx1:idx2+1])
        else:
            parent1[idx2:idx1+1] = reversed(parent1[idx2:idx1+1])
    
    child = parent1
    return child

## Starting From A Greedy Solution (Single Individual)
In the following I will use scramble and inversion as mutations because I feel like others may prove too small of a variation, since we are starting to mutate from just one solution. Inversion may not be enough actually but I'll try.  

Inversion doesn't seem to make sense with inner over, as the chances of mantaining adjacency is high with Inversion.  

Understanding how the various algorithms work takes enough time to not allow me to explore them all, so I'll stick with Inver Over (because it's the one used in the best tsp solution ever recorded) and Cycle (because I dind't manage to study Partially Mapped Crossover too).

### Scramble + Inver Over  

### Scramble + Cycle

### Inversion + Cycle

## Starting From Random Population

### Swap + Cycle

### Swap + Inver Over

### Scramble + Cycle

### Scramble + Inver Over

### Insert + Cycle

### Insert + Inver Over

### Inversion + Cycle

### Inversion + Inver Over

## Full Inver Over (Hypermodern?)

#### Tests

In [361]:
# Checking if the contents of the pickles are the same as the ones in the previous cells
with open(f'{COUNTRY}_tsp_greedy1.pkl', 'rb') as file:
    tsp1 = pickle.load(file)
assert tsp1 == tsp_greedy_1, f"pkl for tsp1 isn't the same as when written"

with open(f'{COUNTRY}_tsp_greedy2.pkl', 'rb') as file:
    tsp2 = pickle.load(file)
assert tsp == tsp_greedy_2, f"pkl for tsp2 isn't the same as when written"

tsp1 = tsp1[:-1]
tsp2 = tsp2[:-1]

# rotate the tsp2 to start at the same city as tsp1
idx = tsp2.index(tsp1[0])
tsp2 = tsp2[idx:] + tsp2[:idx]

# logging.info(f"tsp2 is                      {tsp2}")
# logging.info(f"inversion_mutation(tsp2) is  {inversion_mutation(tsp2, 0.5)}")
# logging.info(f"insert_mutation(tsp2) is     {insert_mutation(tsp2)}")
# logging.info(f"scramble_mutation(tsp2) is   {scramble_mutation(tsp2, 0.5)}")
# logging.info(f"swap_mutation(tsp2) is       {swap_mutation(tsp2)}")

logging.info(f"tsp1 is                                      {tsp1}")
logging.info(f"tsp2 is                                      {tsp2}")
logging.info(f"inver_over_crossover(tsp1, tsp2) is          {inver_over_crossover(tsp1, tsp2)}")
logging.info(f"cycle_crossover(tsp1, tsp2) is               {cycle_crossover(tsp1, tsp2)}")

INFO:root:tsp1 is                                      [0, 33, 12, 30, 9, 4, 19, 32, 25, 28, 18, 20, 3, 6, 44, 45, 23, 43, 41, 5, 40, 22, 42, 13, 16, 29, 10, 26, 39, 34, 15, 14, 21, 35, 11, 1, 2, 38, 17, 31, 8, 37, 24, 7, 36, 27]
INFO:root:tsp2 is                                      [0, 26, 39, 34, 15, 14, 21, 35, 11, 1, 2, 38, 27, 37, 8, 31, 17, 24, 7, 36, 16, 29, 10, 40, 5, 41, 43, 23, 45, 44, 6, 3, 20, 18, 22, 42, 13, 28, 25, 32, 19, 4, 9, 30, 12, 33]
DEBUG:root:city_i: 38 at position 37, city_j: 27 at position 11
INFO:root:inver_over_crossover(tsp1, tsp2) is          [0, 33, 12, 30, 9, 4, 19, 32, 25, 28, 18, 20, 3, 6, 44, 45, 23, 43, 41, 5, 40, 22, 42, 13, 16, 29, 10, 26, 39, 34, 15, 14, 21, 35, 11, 1, 2, 27, 36, 7, 24, 37, 8, 31, 17, 38]
INFO:root:cycle_crossover(tsp1, tsp2) is               [0, 26, 39, 34, 15, 14, 21, 35, 11, 1, 2, 38, 27, 37, 8, 31, 17, 24, 7, 36, 16, 29, 10, 40, 5, 41, 43, 23, 45, 44, 6, 3, 20, 18, 22, 42, 13, 28, 25, 32, 19, 4, 9, 30, 12, 33]
